In [1]:
# ライブラリのインポート
import pandas as pd
import lightgbm as lgb
import warnings
import gc
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
warnings.simplefilter('ignore')

In [2]:
# dataの読み込み
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
submit_df = pd.read_csv('submit_sample.csv',header=None)

In [3]:
# データの量の確認
train_df.shape,test_df.shape,submit_df.shape

((27128, 18), (18083, 17), (18083, 2))

In [4]:
# 訓練データ、テストデータがわかるようにダミーの目的変数を代入
test_df['y']=-999

In [5]:
# 訓練データ、テストデータを結合
all_df = pd.concat([train_df,test_df])
del train_df,test_df
gc.collect()

40

In [6]:
# # カテゴリカラムの前処理
categorical_features = ['job', 'marital', 'education','default','housing','loan','contact','month','poutcome']
# for col in categorical_features:
#     lbl = preprocessing.LabelEncoder()
#     lbl.fit(all_df[col])
#     lbl.transform(all_df[col])
#     all_df[col]=lbl.transform(all_df[col])

In [8]:
all_df = pd.get_dummies(all_df, columns=categorical_features)

In [9]:
# 訓練データ、テストデータの分割
train_df = all_df[all_df['y']!=-999]
test_df = all_df[all_df['y']==-999]

In [10]:
y_train = train_df['y']
X_train = train_df.drop(['y','id'], axis=1)
X_test = test_df.drop(['y','id'], axis=1)

In [11]:
pred_df = pd.DataFrame()

org_X_train = X_train.copy()
org_y_train = y_train.copy()

for i in range(100):
    # 訓練データからデータを分割
    X_train, X_valid, y_train, y_valid = train_test_split(org_X_train, org_y_train, test_size=0.3, random_state=i, stratify=org_y_train)

    # 使用モデルはLGB（パラメータチューニング無）
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
#     lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)
#     lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=categorical_features)

    params = {
        'objective': 'binary'
    }

    model = lgb.train(
        params, lgb_train,
        valid_sets=[lgb_train, lgb_eval],
        verbose_eval=10,
        num_boost_round=1000,
        early_stopping_rounds=10
    )

    y_pred = model.predict(X_test, num_iteration=model.best_iteration)
    pred_df[i] = y_pred

[LightGBM] [Info] Number of positive: 2222, number of negative: 16767
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001068 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1023
[LightGBM] [Info] Number of data points in the train set: 18989, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.117015 -> initscore=-2.021005
[LightGBM] [Info] Start training from score -2.021005
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.237052	valid_1's binary_logloss: 0.24011
[20]	training's binary_logloss: 0.205676	valid_1's binary_logloss: 0.214091
[30]	training's binary_logloss: 0.189687	valid_1's binary_logloss: 0.203555
[40]	training's binary_logloss: 0.178996	valid_1's binary_logloss: 0.1994
[50]	training's binary_logloss: 0.170496	valid_1's binary_logloss: 0.19713
[60

[LightGBM] [Info] Number of positive: 2222, number of negative: 16767
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000843 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 18989, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.117015 -> initscore=-2.021005
[LightGBM] [Info] Start training from score -2.021005
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.232071	valid_1's binary_logloss: 0.246596
[20]	training's binary_logloss: 0.200567	valid_1's binary_logloss: 0.222505
[30]	training's binary_logloss: 0.184404	valid_1's binary_logloss: 0.213353
[40]	training's binary_logloss: 0.173658	valid_1's binary_logloss: 0.210596
[50]	training's binary_logloss: 0.16557	valid_1's binary_logloss: 0.208326


[10]	training's binary_logloss: 0.233554	valid_1's binary_logloss: 0.242397
[20]	training's binary_logloss: 0.201945	valid_1's binary_logloss: 0.217509
[30]	training's binary_logloss: 0.18544	valid_1's binary_logloss: 0.206722
[40]	training's binary_logloss: 0.174633	valid_1's binary_logloss: 0.202244
[50]	training's binary_logloss: 0.166336	valid_1's binary_logloss: 0.200573
[60]	training's binary_logloss: 0.159828	valid_1's binary_logloss: 0.200422
[70]	training's binary_logloss: 0.153997	valid_1's binary_logloss: 0.200188
Early stopping, best iteration is:
[69]	training's binary_logloss: 0.1544	valid_1's binary_logloss: 0.200096
[LightGBM] [Info] Number of positive: 2222, number of negative: 16767
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002496 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 18989, number of used features: 5

[80]	training's binary_logloss: 0.148136	valid_1's binary_logloss: 0.206855
Early stopping, best iteration is:
[74]	training's binary_logloss: 0.151528	valid_1's binary_logloss: 0.206826
[LightGBM] [Info] Number of positive: 2222, number of negative: 16767
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000882 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1022
[LightGBM] [Info] Number of data points in the train set: 18989, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.117015 -> initscore=-2.021005
[LightGBM] [Info] Start training from score -2.021005
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.231748	valid_1's binary_logloss: 0.246704
[20]	training's binary_logloss: 0.201661	valid_1's binary_logloss: 0.224541
[30]	training's binary_logloss: 0.185046

[80]	training's binary_logloss: 0.152255	valid_1's binary_logloss: 0.193268
[90]	training's binary_logloss: 0.147261	valid_1's binary_logloss: 0.193553
Early stopping, best iteration is:
[85]	training's binary_logloss: 0.14984	valid_1's binary_logloss: 0.193162
[LightGBM] [Info] Number of positive: 2222, number of negative: 16767
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001269 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1022
[LightGBM] [Info] Number of data points in the train set: 18989, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.117015 -> initscore=-2.021005
[LightGBM] [Info] Start training from score -2.021005
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.236358	valid_1's binary_logloss: 0.244887
[20]	training's binary_logloss: 0.204374	

...

[LightGBM] [Info] Number of positive: 2222, number of negative: 16767
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000869 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1023
[LightGBM] [Info] Number of data points in the train set: 18989, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.117015 -> initscore=-2.021005
[LightGBM] [Info] Start training from score -2.021005
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.235739	valid_1's binary_logloss: 0.240243
[20]	training's binary_logloss: 0.205852	valid_1's binary_logloss: 0.214824
[30]	training's binary_logloss: 0.189152	valid_1's binary_logloss: 0.203247
[40]	training's binary_logloss: 0.178494	valid_1's binary_logloss: 0.199016
[50]	training's binary_logloss: 0.170153	valid_1's binary_logloss: 0.196588

[LightGBM] [Info] Number of positive: 2222, number of negative: 16767
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000799 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1018
[LightGBM] [Info] Number of data points in the train set: 18989, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.117015 -> initscore=-2.021005
[LightGBM] [Info] Start training from score -2.021005
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.236012	valid_1's binary_logloss: 0.243869
[20]	training's binary_logloss: 0.203172	valid_1's binary_logloss: 0.216735
[30]	training's binary_logloss: 0.187135	valid_1's binary_logloss: 0.205949
[40]	training's binary_logloss: 0.177085	valid_1's binary_logloss: 0.202236
[50]	training's binary_logloss: 0.169053	valid_1's binary_logloss: 0.201075

[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.117015 -> initscore=-2.021005
[LightGBM] [Info] Start training from score -2.021005
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.234328	valid_1's binary_logloss: 0.245563
[20]	training's binary_logloss: 0.20259	valid_1's binary_logloss: 0.220818
[30]	training's binary_logloss: 0.185509	valid_1's binary_logloss: 0.209849
[40]	training's binary_logloss: 0.175043	valid_1's binary_logloss: 0.206131
[50]	training's binary_logloss: 0.167068	valid_1's binary_logloss: 0.204273
[60]	training's binary_logloss: 0.160314	valid_1's binary_logloss: 0.204101
Early stopping, best iteration is:
[57]	training's binary_logloss: 0.162047	valid_1's binary_logloss: 0.203909
[LightGBM] [Info] Number of positive: 2222, number of negative: 16767
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000825 seconds.
You can set `force_row_wise=true` to remove the overhead.
And 

[LightGBM] [Info] Number of positive: 2222, number of negative: 16767
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000773 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 18989, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.117015 -> initscore=-2.021005
[LightGBM] [Info] Start training from score -2.021005
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.232541	valid_1's binary_logloss: 0.247365
[20]	training's binary_logloss: 0.201406	valid_1's binary_logloss: 0.224306
[30]	training's binary_logloss: 0.184904	valid_1's binary_logloss: 0.214471
[40]	training's binary_logloss: 0.17446	valid_1's binary_logloss: 0.210719
[50]	training's binary_logloss: 0.166816	valid_1's binary_logloss: 0.209174


[80]	training's binary_logloss: 0.146228	valid_1's binary_logloss: 0.206583
[90]	training's binary_logloss: 0.141293	valid_1's binary_logloss: 0.20692
Early stopping, best iteration is:
[80]	training's binary_logloss: 0.146228	valid_1's binary_logloss: 0.206583
[LightGBM] [Info] Number of positive: 2222, number of negative: 16767
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000835 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1023
[LightGBM] [Info] Number of data points in the train set: 18989, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.117015 -> initscore=-2.021005
[LightGBM] [Info] Start training from score -2.021005
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.237007	valid_1's binary_logloss: 0.239776
[20]	training's binary_logloss: 0.204416	

In [12]:
pred_df.mean(axis=1)

0        0.634824
1        0.592281
2        0.000872
3        0.001357
4        0.104569
           ...   
18078    0.001817
18079    0.061834
18080    0.001023
18081    0.004060
18082    0.009116
Length: 18083, dtype: float64

In [13]:
submit_df[1] = pred_df.mean(axis=1)

In [14]:
submit_df.to_csv('20200929_001.csv', index=None, header=None)